In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from joblib import Parallel, delayed
from gen_data import gen_data_complex
from proximal import proximal_direct_effect, ProximalDE

#### Verifying we get the same as ivreg on the dataset in the documentation of ivreg

In [3]:
from tests.utilities import gen_kmenta_data
Z, X, Y, labels, controls = gen_kmenta_data()

In [4]:
from ivtests import weakiv_tests
weakiv_tests(Z, X, Y, controls=controls)

(2,
 16,
 array([88.02512828]),
 array([2.3208161e-09]),
 array([113.8720112]),
 array([3.44737137e-10]),
 array([115.65188314]),
 array([1.80718541]),
 array([19.76245769]))

In [116]:
from diagnostics import IVDiagnostics
diag = IVDiagnostics(add_constant=True).fit(Z, X, Y)